In [102]:
import numpy as np
import pandas as pd
from random import shuffle, sample
import stanza
from bidi.algorithm import get_display

In [149]:
eggpath = '/Users/pabloherrero/Documents/ManHatTan/mht/data/processed/EGGs/hebrew_db_team_מפלצת.eggs'
lippath = '/Users/pabloherrero/Documents/ManHatTan/mht/data/processed/LIPSTICK/hebrew_db.lip'
egg = pd.read_csv(eggpath)
egg = egg.set_index('word_ul', drop=False)

lipstick = pd.read_csv(lippath)
lipstick = lipstick.set_index('word_ul', drop=False)
lipstick

,p_recall,n_id,timestamp,delta,user_id,learning_language,ui_language,word_ll,word_ul,lexeme_string,...,mdt_history,mdt_correct,mrt_history,mrt_correct,wdt_history,wdt_correct,wrt_history,wrt_correct,speed,rebag
word_ul,,,,,,,,,,,,,,,,,,,,,
scholarship,0.714,190,1743319778,0,pablo,iw,en,מלגה,scholarship,מלגה/מלגה<NOUN>Fem|Sing|,...,5,5,0,0,2,0,0,0,0.1459,True
intention,0.857,98,1744291702,971924,pablo,iw,en,כונה,intention,כונה/כונה<VERB>Masc|PUAL|Sing|3|Past|Pass|,...,4,4,0,0,2,2,0,0,0.0835,True
towel,1.000,501,1744182216,862438,pablo,iw,en,מגבת,towel,"מגבת/גיבת<VERB>Fem|PAAL|Sing|1,2,3|Part|Act|",...,2,2,0,0,4,4,0,0,0.1115,True
to arrive,0.950,280,1747120372,3800594,pablo,iw,en,להגיע,to arrive,להגיע/הגיע<VERB>HIFIL|Inf|Act|,...,13,12,0,0,5,5,0,0,0.1518,True
Grade,1.000,390,1744122447,802669,pablo,iw,en,ציון,Grade,ציון/ציון<PROPN>,...,6,6,0,0,0,0,0,0,0.1102,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Army,0.000,90,1746629700,0,pablo,iw,en,צבא,Army,צבא/צבא<NOUN>Masc|Sing|,...,0,0,0,0,0,0,0,0,0.0000,False
Cute,0.000,572,1746629700,0,pablo,iw,en,חמוד,Cute,חמוד/חמוד<ADJ>Masc|Sing|,...,0,0,0,0,0,0,0,0,0.0000,False
you asked,0.000,163,1746629700,0,pablo,iw,en,ביקשת,you asked,ביקשת/ביקש<VERB>Masc|PIEL|Sing|1|Past|Act|,...,0,0,0,0,0,0,0,0,0.0000,False


In [66]:
def get_random_nid(lipstick):
    """Get a random n_id from the available ones in stage 0"""
    # Ensure that lipstick is initialized
    if lipstick.empty:
        raise ValueError("Lipstick DataFrame is empty. Please initialize it first.")
    
    # Get the path to the stage 0 n_ids
    pathnid = '/Users/pabloherrero/Documents/ManHatTan/mht/gui/Graphics/index_stage_0.csv'
    stage0_nids = pd.read_csv(pathnid, index_col=None).T.values[0]
    available_nids = np.setdiff1d(stage0_nids, lipstick['n_id'].values)
    random_nid = sample(list(available_nids), 1)[0]
    
    return random_nid

In [174]:
def get_single_lexeme(lip):
    """Get lexeme for a single LIPSTICK entry"""

    lang = lip['learning_language']
    token = lip['word_ll']

    if lang == 'iw':
        lang = 'he'

    nlp = stanza.Pipeline(lang=lang, processors='tokenize,pos,lemma,depparse', tokenize_pretokenized=True)
    # nlp = stanza.Pipeline(lang=lang, processors='tokenize,mwt,pos,lemma,depparse')
    print(f'Token: {token}')
    print(f'Type token[0]: {type(token)}')

    doc = nlp(token)
    for sent in doc.sentences:
        lexeme_string = ''
        # print(len(sent.words))
        for word in sent.words:
            lexeme_string += word.text 
            if word.lemma:
                lexeme_string += '/' + word.lemma 
            if word.upos:
                lexeme_string += '<' + word.upos + '>'
            if word.feats:
                feats = word.feats.split('|')
                lexeme_string += ''.join(feat.split('=')[1] +'|' for feat in feats)
            if word.deprel != 'root':
                lexeme_string +=  ',' + word.deprel
    return lexeme_string

In [170]:
def init_hatched_egg(egg : pd.DataFrame, word_ul : str, lipstick: pd.DataFrame, flag_lexeme = False):
    """Initialize a new entry in the egg dataframe for a specific word."""
    egg_entry = egg.loc[word_ul].copy()
    
    random_nid = get_random_nid(lipstick)
    egg_entry['n_id'] = random_nid

    egg_entry['session_seen'] = 0
    egg_entry['session_correct'] = 0
    egg_entry['p_pred'] = 0
    egg_entry['rebag'] = False
    if flag_lexeme:
        lexeme = get_single_lexeme(egg_entry)
        egg_entry['lexeme_string'] = lexeme
    else:
        egg_entry['lexeme_string'] = 'No lexeme'
    return egg_entry

In [166]:
# self.word_ul
word_ul = 'cat'

egg_entry = init_hatched_egg(egg, word_ul, lipstick=lipstick, flag_lexeme=True)

2025-06-12 19:00:58 WARNING: Language he package default expects mwt, which has been added
2025-06-12 19:00:58 INFO: Loading these models for language: he (Hebrew):
| Processor | Package |
-----------------------
| tokenize  | htb     |
| mwt       | htb     |
| pos       | htb     |
| lemma     | htb     |
| depparse  | htb     |

2025-06-12 19:00:58 INFO: Use device: cpu
2025-06-12 19:00:58 INFO: Loading: tokenize
2025-06-12 19:00:58 INFO: Loading: mwt
/Users/pabloherrero/Library/Python/3.9/lib/python/site-packages/stanza/models/mwt/trainer.py:141: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits 

Type token[0]: <class 'str'>


In [179]:
def add_egg_to_lipstick(egg_entry, lipstick):
    """Add a new egg entry to the lipstick DataFrame."""
    # Ensure that egg_entry is a Series with the correct index
    if not isinstance(egg_entry, pd.Series):
        raise ValueError("egg_entry must be a pandas Series.")
    # Check if the word_ul exists in lipstick
    if egg_entry['word_ul'] in lipstick.index:
        raise ValueError(f"Word '{egg_entry['word_ul']}' already exists in lipstick DataFrame.")
    # Add the egg entry to the lipstick DataFrame
    
    lipstick = lipstick.set_index('word_ul', drop=False)
    new_lipstick = pd.concat([lipstick, egg_entry.to_frame().T])
    # Reset the index to ensure word_ul is the index
    new_lipstick = new_lipstick.reset_index(drop=True)

    return new_lipstick

In [175]:
get_single_lexeme(egg_entry)

2025-06-12 19:02:17 WARNING: Language he package default expects mwt, which has been added
2025-06-12 19:02:17 INFO: Loading these models for language: he (Hebrew):
| Processor | Package |
-----------------------
| tokenize  | htb     |
| mwt       | htb     |
| pos       | htb     |
| lemma     | htb     |
| depparse  | htb     |

2025-06-12 19:02:17 INFO: Use device: cpu
2025-06-12 19:02:17 INFO: Loading: tokenize
2025-06-12 19:02:17 INFO: Loading: mwt
2025-06-12 19:02:17 INFO: Loading: pos
2025-06-12 19:02:17 INFO: Loading: lemma
2025-06-12 19:02:17 INFO: Loading: depparse
2025-06-12 19:02:17 INFO: Done loading processors!


Token: חתול
Type token[0]: <class 'str'>


'חתול/חתול<NOUN>Masc|Sing|'

In [ ]:
# new_lipstick.to_csv(lippath, index=False)

In [203]:
def test_get_random_nid():
    """Test the get_random_nid function."""
    random_nid = get_random_nid(lipstick)
    assert isinstance(random_nid, np.int64), "Random n_id should be an integer."
    assert random_nid not in lipstick['n_id'].values, "Random n_id should not be in the lipstick DataFrame."

def test_init_hatched_egg():
    """Test the init_hatched_egg function."""
    word_ul = 'cat'
    egg_entry = init_hatched_egg(egg, word_ul, lipstick=lipstick, flag_lexeme=True)
    
    assert egg_entry['word_ul'] == word_ul, "Word_ul should match the input word."
    assert 'n_id' in egg_entry, "Egg entry should contain n_id."
    assert 'session_seen' in egg_entry, "Egg entry should contain session_seen."
    assert 'session_correct' in egg_entry, "Egg entry should contain session_correct."
    assert 'p_pred' in egg_entry, "Egg entry should contain p_pred."
    assert 'rebag' in egg_entry, "Egg entry should contain rebag."
    assert 'lexeme_string' in egg_entry, "Egg entry should contain lexeme_string."
    assert isinstance(egg_entry['lexeme_string'], str), "Lexeme string should be a string."

def test_add_egg_to_lipstick():
    """Test the add_egg_to_lipstick function."""
    egg_entry = init_hatched_egg(egg, 'cat', lipstick=lipstick, flag_lexeme=True)
    new_lipstick = add_egg_to_lipstick(egg_entry, lipstick)
    
    assert 'cat' in new_lipstick['word_ul'].values, "New word should be added to lipstick DataFrame."
    assert new_lipstick.shape[0] == lipstick.shape[0] + 1, "New lipstick DataFrame should have one more row than the original."
    assert new_lipstick.loc[new_lipstick['word_ul'] == 'cat', 'n_id'].values[0] == egg_entry['n_id'], "n_id should match the egg entry."

def test_get_single_lexeme():
    """Test the get_single_lexeme function."""
    lip_entry = lipstick.iloc[0]  # Get the first entry in the lipstick DataFrame
    lexeme_string = get_single_lexeme(lip_entry)
    
    assert isinstance(lexeme_string, str), "Lexeme string should be a string."
    assert len(lexeme_string) > 0, "Lexeme string should not be empty."
    print(f"Lexeme string for {lip_entry['word_ll']}: {get_display(lexeme_string)}")

def run_tests():
    """Run all tests."""
    test_get_random_nid()
    test_init_hatched_egg()
    test_add_egg_to_lipstick()
    print("All tests passed!")

2025-06-12 19:14:38 WARNING: Language he package default expects mwt, which has been added
2025-06-12 19:14:38 INFO: Loading these models for language: he (Hebrew):
| Processor | Package |
-----------------------
| tokenize  | htb     |
| mwt       | htb     |
| pos       | htb     |
| lemma     | htb     |
| depparse  | htb     |

2025-06-12 19:14:38 INFO: Use device: cpu
2025-06-12 19:14:38 INFO: Loading: tokenize
2025-06-12 19:14:38 INFO: Loading: mwt
/Users/pabloherrero/Library/Python/3.9/lib/python/site-packages/stanza/models/mwt/trainer.py:141: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits 

Token: חתול
Type token[0]: <class 'str'>


2025-06-12 19:14:39 INFO: Loading: lemma
/Users/pabloherrero/Library/Python/3.9/lib/python/site-packages/stanza/models/lemma/trainer.py:201: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experiment

Token: חתול
Type token[0]: <class 'str'>
All tests passed!
